In [1]:
# import via notebook
!mongoimport --type json -d wine -c ratings --drop --jsonArray ../Resources/clean_wine_data_final.json

2024-05-07T19:50:44.609-0700	connected to: mongodb://localhost/
2024-05-07T19:50:44.609-0700	dropping: wine.ratings
2024-05-07T19:50:46.705-0700	77931 document(s) imported successfully. 0 document(s) failed to import.


In [2]:
# Importing Necessary Libraries
from pymongo import MongoClient
from pprint import pprint
import pandas as pd
import tensorflow as tf
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
%matplotlib inline

In [3]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [4]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [5]:
db = mongo['wine']
db.list_collection_names()

['ratings']

In [6]:
#assign the collection to a variable
wine_df = db['ratings']

In [7]:
cursor = db["ratings"].find({})
json_data = list(cursor)

In [8]:
wine_df = pd.DataFrame(json_data)
wine_df.head()

,_id,country,description,points,price,province,region,title,variety,winery,rating_category,type,vintage
0,663ae8840a70a05d2e5496fb,France,This has great depth of flavor with its fresh ...,87,27,Alsace,Alsace,Jean-Baptiste Adam 2012 Les Natures Pinot Gris...,Pinot Gris,Jean-Baptiste Adam,Good,White,2012
1,663ae8840a70a05d2e5496fc,US,"Slightly reduced, this wine offers a chalky, t...",87,34,California,Alexander Valley,Louis M. Martini 2012 Cabernet Sauvignon (Alex...,Cabernet Sauvignon,Louis M. Martini,Good,Red,2012
2,663ae8840a70a05d2e5496fd,US,Building on 150 years and six generations of w...,87,12,California,Central Coast,Mirassou 2012 Chardonnay (Central Coast),Chardonnay,Mirassou,Good,White,2012
3,663ae8840a70a05d2e5496fe,US,"Soft, supple plum envelopes an oaky structure ...",87,19,California,Napa Valley,Kirkland Signature 2011 Mountain Cuvée Caberne...,Cabernet Sauvignon,Kirkland Signature,Good,Red,2011
4,663ae8840a70a05d2e5496ff,Argentina,Raw black-cherry aromas are direct and simple ...,87,13,Mendoza Province,Mendoza,Gaucho Andino 2011 Winemaker Selection Malbec ...,Malbec,Gaucho Andino,Good,Red,2011


In [9]:
# Drop the non-beneficial ID columns
wine_df = wine_df.drop(columns = ['_id','description','title', 'winery', 'region'])
wine_df.head()

,country,points,price,province,variety,rating_category,type,vintage
0,France,87,27,Alsace,Pinot Gris,Good,White,2012
1,US,87,34,California,Cabernet Sauvignon,Good,Red,2012
2,US,87,12,California,Chardonnay,Good,White,2012
3,US,87,19,California,Cabernet Sauvignon,Good,Red,2011
4,Argentina,87,13,Mendoza Province,Malbec,Good,Red,2011


In [10]:
wine_df['target'] =wine_df['points']>=90
wine_df.tail()

,country,points,price,province,variety,rating_category,type,vintage,target
77926,US,90,22,California,Zinfandel,Very Good,Red,2011,True
77927,France,90,32,Alsace,Pinot Gris,Very Good,White,2012,True
77928,France,90,28,Alsace,Pinot Gris,Very Good,White,2013,True
77929,US,90,75,Oregon,Pinot Noir,Very Good,Red,2004,True
77930,France,90,57,Alsace,Pinot Gris,Very Good,White,2010,True


In [11]:
wine_df = wine_df.drop(columns = ['points','rating_category', 'province', 'type'])

In [12]:
wine_df.head()

,country,price,variety,vintage,target
0,France,27,Pinot Gris,2012,False
1,US,34,Cabernet Sauvignon,2012,False
2,US,12,Chardonnay,2012,False
3,US,19,Cabernet Sauvignon,2011,False
4,Argentina,13,Malbec,2011,False


In [13]:
wine_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77931 entries, 0 to 77930
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   country  77931 non-null  object
 1   price    77931 non-null  int64 
 2   variety  77931 non-null  object
 3   vintage  77931 non-null  object
 4   target   77931 non-null  bool  
dtypes: bool(1), int64(1), object(3)
memory usage: 2.5+ MB


In [15]:
wine_df['vintage'] = wine_df['vintage'].replace('No Year', pd.NA)


,country,price,variety,vintage,target
0,France,27,Pinot Gris,2012,False
1,US,34,Cabernet Sauvignon,2012,False
2,US,12,Chardonnay,2012,False
3,US,19,Cabernet Sauvignon,2011,False
4,Argentina,13,Malbec,2011,False


In [18]:
wine_df.dropna()

,country,price,variety,vintage,target
0,France,27,Pinot Gris,2012,False
1,US,34,Cabernet Sauvignon,2012,False
2,US,12,Chardonnay,2012,False
3,US,19,Cabernet Sauvignon,2011,False
4,Argentina,13,Malbec,2011,False
...,...,...,...,...,...
77926,US,22,Zinfandel,2011,True
77927,France,32,Pinot Gris,2012,True
77928,France,28,Pinot Gris,2013,True
77929,US,75,Pinot Noir,2004,True


In [19]:
# Convert categorical data to numeric with `pd.get_dummies`
wine_dummies_df = pd.get_dummies(wine_df, dtype = int)
wine_dummies_df.head()

,price,target,country_Argentina,country_Australia,country_Canada,country_France,country_Italy,country_Spain,country_US,variety_Bordeaux-style Red Blend,...,vintage_2008,vintage_2009,vintage_2010,vintage_2011,vintage_2012,vintage_2013,vintage_2014,vintage_2015,vintage_2016,vintage_2017
0,27,False,0,0,0,1,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
1,34,False,0,0,0,0,0,0,1,0,...,0,0,0,0,1,0,0,0,0,0
2,12,False,0,0,0,0,0,0,1,0,...,0,0,0,0,1,0,0,0,0,0
3,19,False,0,0,0,0,0,0,1,0,...,0,0,0,1,0,0,0,0,0,0
4,13,False,1,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0


In [20]:
# Split our preprocessed data into our features and target arrays
y = wine_dummies_df['target']
X = wine_dummies_df.drop (columns = 'target')

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [21]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [22]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=80, activation="relu", input_dim=X_train_scaled.shape[1]))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=30, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 80)                5520      
                                                                 
 dense_1 (Dense)             (None, 30)                2430      
                                                                 
 dense_2 (Dense)             (None, 1)                 31        
                                                                 
Total params: 7,981
Trainable params: 7,981
Non-trainable params: 0
_________________________________________________________________


In [23]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [24]:
# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100


2024-05-07 20:13:47.910796: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


1827/1827 [==============================] - 1s 515us/step - loss: 0.5391 - accuracy: 0.7323
Epoch 2/100
1827/1827 [==============================] - 1s 539us/step - loss: 0.5047 - accuracy: 0.7510
Epoch 3/100
1827/1827 [==============================] - 1s 519us/step - loss: 0.4986 - accuracy: 0.7546
Epoch 4/100
1827/1827 [==============================] - 1s 477us/step - loss: 0.4963 - accuracy: 0.7548
Epoch 5/100
1827/1827 [==============================] - 1s 469us/step - loss: 0.4935 - accuracy: 0.7549
Epoch 6/100
1827/1827 [==============================] - 1s 470us/step - loss: 0.4911 - accuracy: 0.7574
Epoch 7/100
1827/1827 [==============================] - 1s 470us/step - loss: 0.4905 - accuracy: 0.7570
Epoch 8/100
1827/1827 [==============================] - 1s 475us/step - loss: 0.4887 - accuracy: 0.7575
Epoch 9/100
1827/1827 [==============================] - 1s 484us/step - loss: 0.4878 - accuracy: 0.7584
Epoch 10/100
1827/1827 [==============================] - 1s 492us/

In [25]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

609/609 - 0s - loss: 0.5046 - accuracy: 0.7516 - 260ms/epoch - 427us/step
Loss: 0.5046156644821167, Accuracy: 0.751629650592804


# Optimization

In [26]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=80, activation="relu", input_dim=X_train_scaled.shape[1]))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=30, activation="relu"))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=30, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 80)                5520      
                                                                 
 dense_4 (Dense)             (None, 30)                2430      
                                                                 
 dense_5 (Dense)             (None, 30)                930       
                                                                 
 dense_6 (Dense)             (None, 1)                 31        
                                                                 
Total params: 8,911
Trainable params: 8,911
Non-trainable params: 0
_________________________________________________________________


In [27]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [28]:
# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
1827/1827 [==============================] - 1s 587us/step - loss: 0.5340 - accuracy: 0.7340
Epoch 2/100
1827/1827 [==============================] - 1s 586us/step - loss: 0.5038 - accuracy: 0.7521
Epoch 3/100
1827/1827 [==============================] - 1s 529us/step - loss: 0.4982 - accuracy: 0.7542
Epoch 4/100
1827/1827 [==============================] - 1s 525us/step - loss: 0.4952 - accuracy: 0.7553
Epoch 5/100
1827/1827 [==============================] - 1s 517us/step - loss: 0.4926 - accuracy: 0.7557
Epoch 6/100
1827/1827 [==============================] - 1s 520us/step - loss: 0.4916 - accuracy: 0.7575
Epoch 7/100
1827/1827 [==============================] - 1s 513us/step - loss: 0.4896 - accuracy: 0.7576
Epoch 8/100
1827/1827 [==============================] - 1s 512us/step - loss: 0.4891 - accuracy: 0.7584
Epoch 9/100
1827/1827 [==============================] - 1s 518us/step - loss: 0.4874 - accuracy: 0.7593
Epoch 10/100
1827/1827 [==============================]

In [29]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

609/609 - 0s - loss: 0.5123 - accuracy: 0.7516 - 292ms/epoch - 479us/step
Loss: 0.5122694373130798, Accuracy: 0.7515782713890076


In [31]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=80, activation="tanh", input_dim=X_train_scaled.shape[1]))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=30, activation="relu"))


# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_7 (Dense)             (None, 80)                5520      
                                                                 
 dense_8 (Dense)             (None, 30)                2430      
                                                                 
 dense_9 (Dense)             (None, 1)                 31        
                                                                 
Total params: 7,981
Trainable params: 7,981
Non-trainable params: 0
_________________________________________________________________


In [32]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [33]:
# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
1827/1827 [==============================] - 1s 541us/step - loss: 0.5329 - accuracy: 0.7314
Epoch 2/100
1827/1827 [==============================] - 1s 530us/step - loss: 0.5022 - accuracy: 0.7522
Epoch 3/100
1827/1827 [==============================] - 1s 476us/step - loss: 0.4975 - accuracy: 0.7543
Epoch 4/100
1827/1827 [==============================] - 1s 478us/step - loss: 0.4950 - accuracy: 0.7549
Epoch 5/100
1827/1827 [==============================] - 1s 474us/step - loss: 0.4930 - accuracy: 0.7567
Epoch 6/100
1827/1827 [==============================] - 1s 477us/step - loss: 0.4912 - accuracy: 0.7566
Epoch 7/100
1827/1827 [==============================] - 1s 485us/step - loss: 0.4901 - accuracy: 0.7567
Epoch 8/100
1827/1827 [==============================] - 1s 476us/step - loss: 0.4888 - accuracy: 0.7580
Epoch 9/100
1827/1827 [==============================] - 1s 481us/step - loss: 0.4877 - accuracy: 0.7587
Epoch 10/100
1827/1827 [==============================]

In [34]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

609/609 - 0s - loss: 0.5036 - accuracy: 0.7526 - 245ms/epoch - 402us/step
Loss: 0.5036019086837769, Accuracy: 0.7526048421859741


In [35]:
# Export our model to HDF5 file
nn.save('Wine.h5')